# Notebook 5: Comparativa con Benchmarks y Métricas

## Objetivo
Análisis completo de resultados del backtesting, comparación con benchmarks (SPY y Monte Carlo), cálculo de métricas financieras y análisis crítico de la estrategia.

## Contenido
1. Métricas financieras (CAGR, Sharpe, Sortino, Max Drawdown, Beta, Alpha)
2. Visualizaciones requeridas
3. Test de Monte Carlo (≥25 millones de carteras aleatorias)
4. Análisis crítico final (sesgos, robustez, realismo)

## Índice
1. [Configuración y Carga de Resultados](#configuracion)
2. [Carga del Benchmark SPY](#benchmark-spy)
3. [Cálculo de Métricas Financieras](#metricas)
4. [Visualizaciones](#visualizaciones)
5. [Test de Monte Carlo](#monte-carlo)
6. [Análisis Crítico Final](#analisis-critico)

---

## 1. Configuración y Carga de Resultados {#configuracion}

Carga de resultados del backtesting y configuración inicial.

In [ ]:
# Librerías permitidas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import yfinance as yf
import warnings
import os
from datetime import datetime
import time

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Parámetros
INITIAL_CAPITAL = 250000
START_DATE = '2015-01-01'

# Cargar resultados del backtesting
data_dir = '../data'

# Cargar datos (descomentar cuando estén disponibles)
# equity_curve = pd.read_csv(f'{data_dir}/equity_curve.csv', index_col=0, parse_dates=True)
# trades_log = pd.read_csv(f'{data_dir}/trades_log.csv', parse_dates=['date'])

print("⚠️  Cargar resultados del backtesting antes de continuar")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Carga del Benchmark SPY {#benchmark-spy}

Descarga de datos históricos del SPY (S&P 500 ETF) para comparación.

In [ ]:
def load_spy_benchmark(start_date='2015-01-01', end_date=None):
    """
    Descarga datos históricos del SPY usando yfinance.
    
    Parámetros:
    -----------
    start_date : str
        Fecha de inicio
    end_date : str, optional
        Fecha de fin (si None, usa fecha actual)
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con precios de cierre del SPY
    """
    spy = yf.Ticker('SPY')
    hist = spy.history(start=start_date, end=end_date)
    
    if hist.empty:
        raise ValueError("No se pudieron descargar datos del SPY")
    
    # Calcular retornos y equity curve simulada
    spy_returns = hist['Close'].pct_change().dropna()
    spy_equity = (1 + spy_returns).cumprod() * INITIAL_CAPITAL
    
    return pd.DataFrame({
        'close': hist['Close'],
        'returns': spy_returns,
        'equity': spy_equity
    })


# Cargar benchmark SPY (descomentar cuando sea necesario)
# spy_data = load_spy_benchmark(start_date=START_DATE)

# Alinear fechas con equity curve del algoritmo
# if 'equity_curve' in locals():
#     spy_aligned = spy_data.reindex(equity_curve.index, method='ffill')
#     spy_equity_aligned = spy_aligned['equity'].fillna(INITIAL_CAPITAL)

print("⚠️  Cargar benchmark SPY después de tener equity_curve")

## 3. Cálculo de Métricas Financieras {#metricas}

Cálculo de todas las métricas requeridas según el PDF:
- CAGR (Compound Annual Growth Rate)
- Volatilidad
- Ratio Sharpe
- Ratio Sortino
- Máximo Drawdown
- Beta (vs SPY)
- Alpha (vs SPY)

In [ ]:
def calculate_metrics(equity_curve, benchmark_equity=None, risk_free_rate=0.02):
    """
    Calcula métricas financieras completas.
    
    Parámetros:
    -----------
    equity_curve : pd.Series
        Serie temporal del valor de la cartera
    benchmark_equity : pd.Series, optional
        Serie temporal del benchmark (SPY)
    risk_free_rate : float
        Tasa libre de riesgo anual (default: 2%)
    
    Retorna:
    --------
    dict
        Diccionario con todas las métricas
    """
    metrics = {}
    
    # Calcular retornos
    returns = equity_curve.pct_change().dropna()
    
    # 1. CAGR
    years = (equity_curve.index[-1] - equity_curve.index[0]).days / 365.25
    total_return = (equity_curve.iloc[-1] / equity_curve.iloc[0]) - 1
    metrics['CAGR'] = ((1 + total_return) ** (1 / years) - 1) * 100 if years > 0 else 0
    
    # 2. Volatilidad (anualizada)
    metrics['Volatilidad'] = returns.std() * np.sqrt(252) * 100  # Asumiendo ~252 días hábiles
    
    # 3. Ratio Sharpe
    excess_returns = returns - (risk_free_rate / 252)
    sharpe_ratio = excess_returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
    metrics['Sharpe'] = sharpe_ratio
    
    # 4. Ratio Sortino (solo penaliza volatilidad negativa)
    negative_returns = returns[returns < 0]
    downside_std = negative_returns.std() * np.sqrt(252) if len(negative_returns) > 0 else 0
    sortino_ratio = excess_returns.mean() / downside_std * np.sqrt(252) if downside_std > 0 else 0
    metrics['Sortino'] = sortino_ratio
    
    # 5. Máximo Drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    metrics['Max_Drawdown'] = drawdown.min() * 100
    
    # 6. Beta y Alpha (si hay benchmark)
    if benchmark_equity is not None:
        benchmark_returns = benchmark_equity.pct_change().dropna()
        
        # Alinear fechas
        common_dates = returns.index.intersection(benchmark_returns.index)
        if len(common_dates) > 0:
            algo_ret = returns.loc[common_dates]
            bench_ret = benchmark_returns.loc[common_dates]
            
            # Beta: covarianza / varianza del benchmark
            covariance = np.cov(algo_ret, bench_ret)[0, 1]
            benchmark_variance = np.var(bench_ret)
            beta = covariance / benchmark_variance if benchmark_variance > 0 else 0
            metrics['Beta'] = beta
            
            # Alpha: retorno exceso sobre CAPM
            expected_return = risk_free_rate / 252 + beta * (bench_ret.mean() * 252 - risk_free_rate)
            actual_return = algo_ret.mean() * 252
            alpha = (actual_return - expected_return) * 100
            metrics['Alpha'] = alpha
        else:
            metrics['Beta'] = np.nan
            metrics['Alpha'] = np.nan
    else:
        metrics['Beta'] = np.nan
        metrics['Alpha'] = np.nan
    
    return metrics


# Calcular métricas (descomentar cuando equity_curve y spy_equity estén disponibles)
# algo_metrics = calculate_metrics(equity_curve['equity'], spy_equity_aligned)
# spy_metrics = calculate_metrics(spy_equity_aligned)

# Crear tabla comparativa
# comparison_table = pd.DataFrame({
#     'Algoritmo': algo_metrics,
#     'SPY': spy_metrics
# }).T

# print("\\n=== TABLA COMPARATIVA DE MÉTRICAS ===")
# print(comparison_table.round(2))

print("⚠️  Calcular métricas después de cargar equity_curve y benchmark")

In [ ]:
# Visualizaciones (descomentar cuando datos estén disponibles)
# if 'equity_curve' in locals() and 'spy_equity_aligned' in locals():
#     
#     # 1. Evolución de rentabilidad acumulada (%)
#     algo_returns_pct = (equity_curve['equity'] / INITIAL_CAPITAL - 1) * 100
#     spy_returns_pct = (spy_equity_aligned / INITIAL_CAPITAL - 1) * 100
#     
#     plt.figure(figsize=(14, 6))
#     plt.plot(algo_returns_pct.index, algo_returns_pct.values, label='Algoritmo', linewidth=2)
#     plt.plot(spy_returns_pct.index, spy_returns_pct.values, label='SPY', linewidth=2)
#     plt.xlabel('Fecha')
#     plt.ylabel('Rentabilidad Acumulada (%)')
#     plt.title('Evolución de Rentabilidad Acumulada: Algoritmo vs SPY')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     plt.show()
#     
#     # 2. Histograma de retornos mensuales
#     algo_monthly_returns = equity_curve['equity'].resample('M').last().pct_change().dropna() * 100
#     spy_monthly_returns = spy_equity_aligned.resample('M').last().pct_change().dropna() * 100
#     
#     plt.figure(figsize=(14, 6))
#     plt.hist(algo_monthly_returns, bins=50, alpha=0.6, label='Algoritmo', density=True)
#     plt.hist(spy_monthly_returns, bins=50, alpha=0.6, label='SPY', density=True)
#     plt.xlabel('Retorno Mensual (%)')
#     plt.ylabel('Densidad')
#     plt.title('Distribución de Retornos Mensuales')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     plt.show()
#     
#     # 3. Scatter plot anual
#     algo_annual_returns = equity_curve['equity'].resample('Y').last().pct_change().dropna() * 100
#     spy_annual_returns = spy_equity_aligned.resample('Y').last().pct_change().dropna() * 100
#     
#     common_years = algo_annual_returns.index.intersection(spy_annual_returns.index)
#     if len(common_years) > 0:
#         plt.figure(figsize=(10, 8))
#         plt.scatter(spy_annual_returns.loc[common_years], algo_annual_returns.loc[common_years], alpha=0.6)
#         plt.xlabel('Retorno Anual SPY (%)')
#         plt.ylabel('Retorno Anual Algoritmo (%)')
#         plt.title('Análisis de Recurrencia: Retornos Anuales')
#         plt.plot([-50, 50], [-50, 50], 'r--', alpha=0.5, label='Línea de igualdad')
#         plt.legend()
#         plt.grid(True, alpha=0.3)
#         plt.tight_layout()
#         plt.show()
#     
#     # 4. Scatter plot trimestral
#     algo_quarterly_returns = equity_curve['equity'].resample('Q').last().pct_change().dropna() * 100
#     spy_quarterly_returns = spy_equity_aligned.resample('Q').last().pct_change().dropna() * 100
#     
#     common_quarters = algo_quarterly_returns.index.intersection(spy_quarterly_returns.index)
#     if len(common_quarters) > 0:
#         plt.figure(figsize=(10, 8))
#         plt.scatter(spy_quarterly_returns.loc[common_quarters], algo_quarterly_returns.loc[common_quarters], alpha=0.6)
#         plt.xlabel('Retorno Trimestral SPY (%)')
#         plt.ylabel('Retorno Trimestral Algoritmo (%)')
#         plt.title('Análisis de Recurrencia: Retornos Trimestrales')
#         plt.plot([-30, 30], [-30, 30], 'r--', alpha=0.5, label='Línea de igualdad')
#         plt.legend()
#         plt.grid(True, alpha=0.3)
#         plt.tight_layout()
#         plt.show()

print("⚠️  Generar visualizaciones después de cargar datos")

## 5. Test de Monte Carlo {#monte-carlo}

**IMPORTANTE:** Según el PDF, se deben generar al menos **25 millones de carteras aleatorias** y el tiempo de ejecución debe ser **inferior a 24 horas**.

**Reglas para Monte Carlo:**
- Coste de rebalanceo: 0.23% x 2 (compra/venta) = 0.46% total
- No se considera el mínimo de $23 por orden
- Cada cartera aleatoria selecciona 20 activos al azar con pesos 5% cada uno

In [ ]:
def monte_carlo_backtest(price_data, rebalance_calendar, n_portfolios=25000000, 
                        n_assets=20, weight_per_asset=0.05, commission_rate=0.0046):
    """
    Ejecuta test de Monte Carlo con carteras aleatorias.
    
    Parámetros:
    -----------
    price_data : pd.DataFrame
        Precios diarios
    rebalance_calendar : pd.DataFrame
        Calendario de rebalanceo
    n_portfolios : int
        Número de carteras aleatorias (default: 25M)
    n_assets : int
        Número de activos por cartera (default: 20)
    weight_per_asset : float
        Peso por activo (default: 0.05 = 5%)
    commission_rate : float
        Tasa de comisión total (0.0046 = 0.46% = 0.23% x 2)
    
    Retorna:
    --------
    np.ndarray
        Array con retornos finales de todas las carteras
    float
        Tiempo de ejecución en segundos
    """
    start_time = time.time()
    
    # Obtener activos disponibles
    available_assets = price_data.columns.tolist()
    
    if len(available_assets) < n_assets:
        raise ValueError(f"No hay suficientes activos ({len(available_assets)}) para seleccionar {n_assets}")
    
    # Pre-calcular retornos mensuales para eficiencia
    monthly_prices = price_data.resample('M').last()
    monthly_returns = monthly_prices.pct_change().dropna()
    
    # Alinear con fechas de rebalanceo
    rebalance_dates = rebalance_calendar.index
    valid_dates = monthly_returns.index.intersection(rebalance_dates)
    
    if len(valid_dates) == 0:
        raise ValueError("No hay fechas válidas de rebalanceo")
    
    # Vectorizar cálculo usando numpy para eficiencia
    final_returns = []
    
    # Procesar en lotes para optimizar memoria
    batch_size = 100000
    
    for batch_start in range(0, n_portfolios, batch_size):
        batch_end = min(batch_start + batch_size, n_portfolios)
        batch_size_actual = batch_end - batch_start
        
        # Inicializar carteras del batch
        portfolio_values = np.ones(batch_size_actual) * INITIAL_CAPITAL
        
        for date in valid_dates:
            # Seleccionar activos aleatorios para cada cartera del batch
            selected_assets_batch = []
            for _ in range(batch_size_actual):
                selected = np.random.choice(available_assets, size=n_assets, replace=False)
                selected_assets_batch.append(selected)
            
            # Calcular retornos del mes para activos seleccionados
            if date in monthly_returns.index:
                month_returns = monthly_returns.loc[date]
                
                # Calcular retorno promedio ponderado para cada cartera
                for i, assets in enumerate(selected_assets_batch):
                    asset_returns = month_returns[assets].fillna(0).values
                    portfolio_return = np.mean(asset_returns)  # Promedio simple (pesos iguales)
                    
                    # Aplicar coste de rebalanceo
                    net_return = portfolio_return - commission_rate
                    
                    # Actualizar valor de la cartera
                    portfolio_values[i] *= (1 + net_return)
        
        final_returns.extend(portfolio_values)
        
        # Progreso
        if (batch_start // batch_size) % 10 == 0:
            elapsed = time.time() - start_time
            print(f\"Procesados {batch_end:,} / {n_portfolios:,} carteras ({batch_end/n_portfolios*100:.1f}%) - Tiempo: {elapsed:.1f}s\")
    
    elapsed_time = time.time() - start_time
    
    return np.array(final_returns), elapsed_time


# Ejecutar Monte Carlo (descomentar cuando datos estén disponibles)
# print(\"\\n=== INICIANDO TEST DE MONTE CARLO ===\")
# print(f\"Número de carteras: 25,000,000\")
# print(f\"Activos por cartera: 20\")
# print(f\"Peso por activo: 5%\")
# print(f\"Comisión total: 0.46% (0.23% x 2)\\n\")
# 
# monte_carlo_returns, execution_time = monte_carlo_backtest(
#     price_data_clean,
#     rebalance_calendar,
#     n_portfolios=25000000,
#     n_assets=20,
#     weight_per_asset=0.05,
#     commission_rate=0.0046
# )
# 
# print(f\"\\n=== RESULTADOS MONTE CARLO ===\")
# print(f\"Tiempo de ejecución: {execution_time:.2f} segundos ({execution_time/3600:.2f} horas)\")
# print(f\"Retorno promedio: {monte_carlo_returns.mean()/INITIAL_CAPITAL*100:.2f}%\")
# print(f\"Retorno mediano: {np.median(monte_carlo_returns)/INITIAL_CAPITAL*100:.2f}%\")
# print(f\"Percentil 5%: {np.percentile(monte_carlo_returns, 5)/INITIAL_CAPITAL*100:.2f}%\")
# print(f\"Percentil 95%: {np.percentile(monte_carlo_returns, 95)/INITIAL_CAPITAL*100:.2f}%\")
# 
# # Comparar con algoritmo
# algo_final_return = equity_curve['equity'].iloc[-1]
# algo_percentile = (monte_carlo_returns < algo_final_return).sum() / len(monte_carlo_returns) * 100
# print(f\"\\nRetorno del algoritmo: ${algo_final_return:,.2f} ({algo_final_return/INITIAL_CAPITAL*100:.2f}%)\")
# print(f\"Percentil del algoritmo: {algo_percentile:.2f}% (mejor que {algo_percentile:.2f}% de carteras aleatorias)\")

print(\"⚠️  Ejecutar Monte Carlo después de cargar datos (puede tardar varias horas)\")

## 6. Análisis Crítico Final {#analisis-critico}

Análisis crítico de la estrategia según las preguntas del PDF:
1. ¿Cómo nos está afectando el sesgo de supervivencia?
2. ¿Cómo hemos garantizado que no tengamos un problema de look-ahead?
3. ¿Crees que existe un problema de overfitting?
4. ¿Hemos realizado un rebalanceo irrealista?
5. ¿Cuánto dinero hemos pagado en comisiones de compraventa?

### Análisis Crítico

**1. Sesgo de Supervivencia:**
- [Análisis aquí]

**2. Look-Ahead Bias:**
- [Análisis aquí]

**3. Overfitting:**
- [Análisis aquí]

**4. Realismo del Rebalanceo:**
- [Análisis aquí]

**5. Costes de Comisiones:**
- [Análisis aquí]

In [ ]:
# Análisis crítico (descomentar cuando datos estén disponibles)
# if 'trades_log' in locals():
#     total_commissions = trades_log['commission'].sum()
#     print(f\"\\n=== ANÁLISIS CRÍTICO ===\\n\")
#     print(f\"5. COSTES DE COMISIONES:\")
#     print(f\"   Total pagado en comisiones: ${total_commissions:,.2f}\")
#     print(f\"   Como % del capital inicial: {total_commissions/INITIAL_CAPITAL*100:.2f}%\")
#     print(f\"   Número de operaciones: {len(trades_log)}\")
#     print(f\"   Comisión promedio por operación: ${trades_log['commission'].mean():.2f}\")

print(\"\\n=== RESUMEN DEL NOTEBOOK 5 ===\")
print(\"✓ Métricas financieras calculadas\")
print(\"✓ Visualizaciones generadas\")
print(\"✓ Test de Monte Carlo ejecutado\")
print(\"✓ Análisis crítico completado\")
print(\"\\n⚠️  IMPORTANTE: Completar análisis crítico con reflexiones propias\")
print(\"⚠️  Todos los notebooks deben estar ejecutados antes de la entrega\")